In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pickle
from catboost import CatBoostClassifier

In [4]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

with engine.connect() as conn:
    user_df = pd.read_sql(
        sql="SELECT * FROM public.user_data",
        con=conn.connection
    )

with engine.connect() as conn:
    post_df = pd.read_sql(
        sql="SELECT * FROM darja_stiheeva_lms4973_post_features_bert_2",
        con=conn.connection
    )

with engine.connect() as conn:
    feed_df = pd.read_sql(
        sql="""
        SELECT timestamp, user_id, post_id,
        CASE 
            WHEN action = 'like' THEN 1
            ELSE target
        END AS target
        FROM feed_data
        WHERE NOT (action = 'view' AND target = 1)
        LIMIT 5000000;
        """,
        con=conn.connection
    )


C:\Users\HYPERPC\AppData\Local\Temp\ipykernel_3668\3133484240.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_df = pd.read_sql(
C:\Users\HYPERPC\AppData\Local\Temp\ipykernel_3668\3133484240.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  post_df = pd.read_sql(
C:\Users\HYPERPC\AppData\Local\Temp\ipykernel_3668\3133484240.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  feed_df = pd.read_sql(


In [5]:
post_df = post_df.drop('text', axis=1)
post_df

,post_id,topic,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29


In [ ]:
feed_df['timestamp'] = pd.to_datetime(feed_df['timestamp'])

feed_df['hour'] = feed_df['timestamp'].dt.hour
feed_data = feed_df.drop(columns=['timestamp'])

In [ ]:
def merge_data(user_data, post_data, feed_data):

    merged_data = feed_data.merge(user_data, on='user_id', how='left')
    merged_data = merged_data.merge(post_data, on='post_id', how='left')
    
    return merged_data

merged_df = merge_data(user_df, post_df, feed_data)
merged_df = merged_df.drop(columns=['user_id', 'post_id'])

In [ ]:
categorical_features = ['topic', 'hour', 'city', 'country', 'exp_group', 'gender', 'source', 'os']

In [ ]:
X = merged_df.drop(columns=['target'])
all_features = categorical_features + [col for col in X.columns if col not in categorical_features]
X = X[all_features]

y = merged_df['target']

In [ ]:
model = CatBoostClassifier(iterations=100,
                           depth=6,
                           learning_rate=0.1)

model.fit(X, y, categorical_features)

In [ ]:
model.save_model('catboost_model',
                           format="cbm")

from_file = CatBoostClassifier() 

from_file.load_model("catboost_model")

from_file.predict_proba(X)